In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

In [2]:
# importing data after performing EDA and Feature Engineering
train = pd.read_csv("train1.csv")
test = pd.read_csv("test1.csv")

C:\Users\shivam\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (285,288,289,296,300,301,302,303,304,306,307,308,309,310,311,312,313) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\shivam\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (284,287,288,295,299,300,301,302,303,305,306,307,308,309,310,311,312) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
print(train.shape)
print(test.shape)

(354324, 320)
(236216, 319)


In [ ]:
# From Eda we have seen that there are outliers in some of columns

In [4]:
##replacing outliers with median

In [5]:
#fuction to replace outliers with median
def replaceoutliers(x,value): 
    mean = float(train[x].median())
    train[x] = np.where(train[x] > value, mean, train[x])
    mean = float(test[x].median())
    test[x] = np.where(test[x] > value, mean, test[x])  

In [6]:
#fuction to replace outliers with median
def replaceoutliers_less(x,value): 
    mean = float(train[x].median())
    train[x] = np.where(train[x] < value, mean, train[x])
    mean = float(test[x].median())
    test[x] = np.where(test[x] < value, mean, test[x]) 

In [7]:
# for c columns
replaceoutliers('C1',2200)
replaceoutliers('C2',3000)
replaceoutliers('C4',1500)
replaceoutliers('C6',1400)
replaceoutliers('C7',1400)
replaceoutliers('C8',2000)
replaceoutliers('C10',2000)
replaceoutliers('C11',2000)
replaceoutliers('C12',900)

In [8]:
# for d columns
replaceoutliers_less('D4',0)
replaceoutliers_less('D6',0)
replaceoutliers_less('D11',0)
replaceoutliers_less('D12',0)
replaceoutliers_less('D14',0)
replaceoutliers_less('D15',0)

In [9]:
## NULL value Percentage

In [10]:
# calcuating Null value percenatge
total = train.isnull().sum().sort_values(ascending = False)
percent = (train.isnull().sum()/train.isnull().count()*100).sort_values(ascending = False)
missing_train_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_train_data.head(30)

,Total,Percent
id_24,351487,99.199320
id_25,351248,99.131868
id_07,351237,99.128764
id_08,351237,99.128764
id_26,351232,99.127352
id_21,351232,99.127352
id_27,351228,99.126223
id_22,351228,99.126223
id_23,351228,99.126223
Logdist2,331717,93.619681


In [11]:
# replace high percentage null value features which have less interseting results when performed EDA
train.drop(['id_21','id_22','id_23','id_24','id_25','id_26','id_27','id_07','id_08','Logdist2','D7'],axis =1,inplace = True)
test.drop(['id_21','id_22','id_23','id_24','id_25','id_26','id_27','id_07','id_08','Logdist2','D7'],axis =1,inplace = True)

In [12]:
## Checking Skewnwss 

In [13]:
#sort skewness value in decreasing order
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = train.skew(axis = 0, skipna = True)
df.sort_values(ascending=False)

V311                 303.457524
V129                 243.936487
V309                 229.943855
V206                 180.274756
V319                 168.038493
V266                 157.560669
V334                 140.929155
V131                 111.946856
V227                 100.807054
V321                 100.624658
V214                  97.554981
V312                  93.077301
V276                  90.251993
V268                  87.678864
C3                    85.326593
V245                  77.156409
V278                  71.444561
V335                  61.290265
V136                  55.095104
V221                  53.110766
V277                  52.515628
V320                  52.028807
V292                  50.636890
V291                  50.065154
V222                  45.679212
V267                  44.554813
V256                  41.422749
V254                  40.554981
V216                  39.982803
C12                   39.281428
V270                  32.929719
V161    

In [14]:
train.drop(['V311','V129','V309','V206','V319','V266','V334','V131','V227','V321'],axis =1,inplace = True)
test.drop(['V311','V129','V309','V206','V319','V266','V334','V131','V227','V321'],axis =1,inplace = True)

In [15]:
train.shape

(354324, 299)

In [16]:
## Deleting highly correlated features

In [17]:
# dropping correlated d columns
columns = ['D2','D6','D12']
train.drop(columns,inplace = True,axis =1)
test.drop(columns,inplace = True,axis =1)

In [18]:
#dropping correlated c columns
columns = ['C2','C6','C11','C10']
train.drop(columns,inplace = True,axis =1)
test.drop(columns,inplace = True,axis =1)

In [19]:
# less relevant according to EDA
del train['DeviceInfo']
del test['DeviceInfo']

In [20]:
## Handling NULL values by repalcing them with mean,median mode

In [21]:
for i in train.columns:
    if 'M' in i:
        train[i].fillna(train[i].mode()[0],inplace=True)

In [22]:
for i in test.columns:
    if 'M' in i:
        test[i].fillna(test[i].mode()[0],inplace=True)

In [23]:
for i in train.columns:
    if train[i].dtypes=="float64":
        train[i].fillna(train[i].mean(),inplace=True)

In [24]:
for i in test.columns:
    if test[i].dtypes=="float64":
        test[i].fillna(test[i].mean(),inplace=True)

In [25]:
train.fillna(train.select_dtypes(include='object').mode().iloc[0], inplace=True)
test.fillna(test.select_dtypes(include='object').mode().iloc[0], inplace=True)

In [26]:
#Encoding of features using one hot and label--------------

In [27]:
objList = train.select_dtypes(include ="object")
for feat in objList:
    print(feat,"->",train[feat].nunique())

ProductCD -> 5
card4 -> 4
card6 -> 2
M1 -> 2
M2 -> 2
M3 -> 2
M4 -> 3
M5 -> 2
M6 -> 2
M7 -> 2
M8 -> 2
M9 -> 2
id_12 -> 2
id_15 -> 3
id_16 -> 2
id_28 -> 2
id_29 -> 2
id_30 -> 75
id_31 -> 122
id_33 -> 221
id_34 -> 4
id_35 -> 2
id_36 -> 2
id_37 -> 2
id_38 -> 2
DeviceType -> 2
P_parent_emaildomain -> 44
R_parent_emaildomain -> 45


In [28]:
objList = test.select_dtypes(include ="object")
for feat in objList:
    print(feat,"->",test[feat].nunique())

ProductCD -> 5
card4 -> 4
card6 -> 2
M1 -> 2
M2 -> 2
M3 -> 2
M4 -> 3
M5 -> 2
M6 -> 2
M7 -> 2
M8 -> 2
M9 -> 2
id_12 -> 2
id_15 -> 3
id_16 -> 2
id_28 -> 2
id_29 -> 2
id_30 -> 72
id_31 -> 121
id_33 -> 182
id_34 -> 3
id_35 -> 2
id_36 -> 2
id_37 -> 2
id_38 -> 2
DeviceType -> 2
P_parent_emaildomain -> 44
R_parent_emaildomain -> 45


In [29]:
print(train.shape,test.shape)

(354324, 291) (236216, 290)


In [30]:
objList = ['ProductCD','card4','M4']
for feat in objList:
        dummies = pd.get_dummies(train[[feat]])
        train= pd.concat([train, dummies], axis=1)
        del(train[feat])
        dummies = pd.get_dummies(test[[feat]])
        test= pd.concat([test, dummies], axis=1)
        del(test[feat])

In [31]:
print(train.shape,test.shape)

(354324, 300) (236216, 299)


In [32]:
y_train = train['isFraud'].copy()

In [33]:
del train['isFraud']

In [34]:
# label encoding of categorical features
from sklearn import preprocessing
for f in train.columns:
    if train[f].dtype=='object' or test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[f].values) + list(test[f].values))
        train[f] = lbl.transform(list(train[f].values))
        test[f] = lbl.transform(list(test[f].values))  

In [35]:
train['isFraud'] = y_train

In [36]:
#saving files
train.to_csv('trainPreprocessed.csv',index=False)
test.to_csv('testPreprocessed.csv',index=False)

In [37]:
#end------------------------------------